- 2020-08-10
- Google Colab
- TF: 2.3.0
- yolov4: 1.1.0

In [ ]:
!python3 -m pip install yolov4

In [ ]:
!unzip /content/drive/My\ Drive/Hard_Soft/NN/coco/train2017.zip
!unzip /content/drive/My\ Drive/Hard_Soft/NN/coco/val2017.zip

In [ ]:
from tensorflow.keras import callbacks, optimizers
from py_src.yolov4.tf import SaveWeightsCallback, YOLOv4

yolo = YOLOv4(tiny=True)

yolo.classes = "/content/drive/My Drive/Hard_Soft/NN/coco/coco.names"

yolo.input_size = 608
yolo.batch_size = 32

train_data_set = yolo.load_dataset(
    "/content/drive/My Drive/Hard_Soft/NN/coco/train2017.txt",
    image_path_prefix="/content/train2017",
)
val_data_set = yolo.load_dataset(
    "/content/drive/My Drive/Hard_Soft/NN/coco/val2017.txt",
    image_path_prefix="/content/val2017",
    training=False,
)

yolo.make_model(activation1="relu")
yolo.load_weights(
    "/content/drive/My Drive/Hard_Soft/NN/trained/yolov4-tiny.conv.29",
    weights_type="yolo",
)

epochs = 80
lr = 0.0025

optimizer = optimizers.Adam(learning_rate=lr)
yolo.compile(optimizer=optimizer, loss_iou_type="ciou")


def lr_scheduler(epoch):
    if epoch < int(epochs * 0.8):
        return lr
    elif epoch < int(epochs * 0.9):
        return lr * 0.1
    else:
        return lr * 0.01


yolo.fit(
    train_data_set,
    epochs=epochs,
    callbacks=[
        callbacks.LearningRateScheduler(lr_scheduler),
        callbacks.TerminateOnNaN(),
        callbacks.TensorBoard(
            histogram_freq=1,
            log_dir="/content/drive/My Drive/Hard_Soft/NN/logs",
        ),
        SaveWeightsCallback(
            yolo=yolo,
            dir_path="/content/drive/My Drive/Hard_Soft/NN/trained",
            weights_type="yolo",
            epoch_per_save=1,
        ),
    ],
    validation_data=val_data_set,
    validation_steps=20,
    steps_per_epoch=1000,
)

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import time

frame = cv2.imread(val_data_set.dataset[0])
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

start_time = time.time()
bboxes = yolo.predict(frame)
exec_time = time.time() - start_time
print("time: {:.2f} ms".format(exec_time * 1000))

frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
image = yolo.draw_bboxes(frame, bboxes)
cv2_imshow(image)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir /content/drive/My\ Drive/Hard_Soft/NN/logs